In [0]:
df_bronze = spark.table("workspace.bronze.flights_delay_maior_100")
display(df_bronze.limit(20))

In [0]:
from pyspark.sql.functions import col, trim, upper

df_silver = (
    df_bronze
    .dropna(subset=["origin", "destination", "delay"])  # remove linhas nulas
    .withColumn("origin", trim(upper(col("origin"))))   # remove espaços e padroniza texto
    .withColumn("destination", trim(upper(col("destination"))))
    .filter(col("delay") < 2000)                        # remove atrasos absurdos
)
display(df_silver.limit(5))


In [0]:
from pyspark.sql.functions import when

df_silver = df_silver.withColumn(
    "nivel_atraso",
    when(col("delay") < 200, "baixo")
    .when(col("delay") < 500, "moderado")
    .otherwise("alto")
)

In [0]:
df_silver.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("workspace.silver.flights_delay_limpos")


In [0]:
%sql
SELECT COUNT(*) AS total_linhas, AVG(delay) AS media_atraso
FROM workspace.silver.flights_delay_limpos;


In [0]:
%sql
SELECT nivel_atraso, COUNT(*) 
FROM workspace.silver.flights_delay_limpos 
GROUP BY nivel_atraso;